# Part I. ETL Pipeline for Pre-Processing the Files

## This section is to be run first so that the ETL pipeline is run to centralize / denormalize all of our data into a single table

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)  # Skip header
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Designing the right tables in Apache Cassandra for the right queries

## After the ETL pipeline has run, a new CSV file called <font color=red>event_datafile_new.csv</font> is generated in the current directory.  This `event_datafile_new.csv` file contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## We will now begin to create and set the Keyspace on our local machine

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Preamble - print out indices of the header and the columns themselves

In [8]:
# Print out indices of the header and the columns themselves
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    header = next(csvreader) # skip header

for i, h in enumerate(header):
    print(i, h)

0 artist
1 firstName
2 gender
3 itemInSession
4 lastName
5 length
6 level
7 location
8 sessionId
9 song
10 userId


## Query 1

The following is the target query we wish to execute as the first query:

> Give me the artist, song title and song's length in the music app history that was heard during `sessionId = 338` and `itemInSession = 4`

### Step #1 - Create the table and form the query to probe the table

The primary key was chosen to be the session ID as this makes sense to use this as the primary differentiator for each user's session and is guaranteed to be unique.  With each user session, there could be multiple items in session, though of course they don't all appear at the same time and reflect the user's choices in listening to different songs at any given time.  The item in session was chosen as a clustering key.

In [9]:
## First create the table
query = "CREATE TABLE IF NOT EXISTS song_information "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## Query for probing the database once the table is complete
query1 = 'SELECT artist_name, song_title, song_length FROM song_information WHERE session_id = 338 AND item_in_session = 4'                    

### Step #2 - Read in denormalized data one row at a time and populate our table

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Build INSERT query
        query = "INSERT INTO song_information (artist_name, song_title, song_length, session_id, item_in_session) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Extract the elements of each linen in the denormalized CSV and place into table
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

### Step #3 - Do a SELECT to verify that the data have been inserted into each table

Note that I have extracted each row from the `SELECT` statement and have placed them into a Pandas dataframe for easy reading.  Recall that the query was:

> Give me the artist, song title and song's length in the music app history that was heard during `sessionId = 338` and `itemInSession = 4`

In [11]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

df = pd.DataFrame(columns=['artist_name', 'song_title', 'song_length'])
pd.options.display.max_colwidth = 100

for row in rows:
    df = df.append({'artist_name': row.artist_name, 'song_title': row.song_title, 'song_length': row.song_length}, ignore_index=True)
df

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2

The following is the target query we wish to execute as the second query:

> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for `userid = 10`, `sessionid = 182`

### Step #1 - Create the table and form the query to probe the table

The primary key was chosen to be the user ID as this ID is guaranteed to be unique no matter which session we are in.  However, the user can have multiple sessions throughout the day, week, etc. so to further differentiate the time instance that the user was listening to a particular song, we further differentiate by session ID.  These two together should form a composite key to isolate out the particular time instance the user began his or her session thus forming our composite key.  To ensure we extract out the songs want to look at based on their unique session ID, we would like to sort the songs played during this session by `item_in_session`, so this leads using this as our clustering key.  Note that the user attribute is not required as a clustering key since the previous information up to this point is give us the necessary content we need.

In [12]:
## First create the table
query = "CREATE TABLE IF NOT EXISTS user_listening_history "
query = query + "( user_id int, session_id int, item_in_session int, artist_name text, song_title text, user text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## Query for probing the database once the table is complete
query2 = 'SELECT artist_name, song_title, user, item_in_session FROM user_listening_history WHERE user_id = 10 AND session_id = 182'                    

### Step #2 - Read in denormalized data one row at a time and populate our table

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listening_history (artist_name, song_title, user, user_id, session_id, item_in_session) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1] + ' ' + line[4], int(line[10]), int(line[8]), int(line[3])))

### Step #3 - Do a SELECT to verify that the data have been inserted into each table

Note that I have extracted each row from the `SELECT` statement and have placed them into a Pandas dataframe for easy reading.  Recall that the query was:

> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for `userid = 10`, `sessionid = 182`

In [14]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
df = pd.DataFrame(columns=['artist_name', 'song_title', 'user'])
for row in rows:
    df = df.append({'artist_name': row.artist_name, 'song_title': row.song_title, 'user': row.user}, ignore_index=True)
df

,artist_name,song_title,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie Cruz


## Query 3

The following is the target query we wish to execute as the second query:

> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Step #1 - Create the table and form the query to probe the table

The primary key was chosen to be the song title and the clustering key was chosen to be the user name.  The reason why is because just cataloguing the information by using the song title alone does not take into account the multitude of users who could listen to the same song, which would mean that we would only ever get one row of a retrieved result.  Our desire is to find *all* users who listened to a particular song so we first partition the data based on the song that was listened to, then further decompose our data based on which users listened to which songs.

In [15]:
## First create the table
query = "CREATE TABLE IF NOT EXISTS list_users_for_song "
query = query + "(song_title text, user text, PRIMARY KEY (song_title, user))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## Query for probing the database once the table is complete
query3 = "SELECT song_title, user FROM list_users_for_song WHERE song_title = 'All Hands Against His Own'"
                    

### Step #2 - Read in denormalized data one row at a time and populate our table

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO list_users_for_song (song_title, user) "
        query = query + "VALUES (%s, %s)"
        session.execute(query, (line[9], line[1] + ' ' + line[4]))

### Step #3 - Do a SELECT to verify that the data have been inserted into each table

Note that I have extracted each row from the `SELECT` statement and have placed them into a Pandas dataframe for easy reading.  Recall that the query was:

> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
df = pd.DataFrame(columns=['user'])
for row in rows:
    df = df.append({'user': row.user}, ignore_index=True)
df

,user
0,Jacqueline Lynch
1,Sara Johnson
2,Tegan Levine


## Drop the tables before closing out the sessions

Now that we're finished with our three queries for this assignment, let's drop all of the tables for cleanup

In [18]:
query = 'DROP TABLE IF EXISTS song_information'
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = 'DROP TABLE IF EXISTS user_listening_history'
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = 'DROP TABLE IF EXISTS list_users_for_song'
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()